In [ ]:
```
데이콘 변수 설명

Gender : 전복 성별
Lenght : 전복 길이
Diameter : 전복 둘레
Height : 전복 키 
Whole : Weight : 전복 전체 무게
Shucked Weight : 껍질을 제외한 무게
Viscra Weight : 내장 무게
Shell Weight : 껍질 무게
Target : 전복 나이
-----------------------------------------------------

※변수에 대한 다른 생각

Lenght / 연속 / mm / 가장 긴 껍질 측정 (장축)
Diameter / 연속 / mm / 길이에 수직 (직경 / 단축)
Height / 연속 / mm / 높이 , 껍질에 고기 포함
Whole Weight : 전복 전체 무게 (whole abalone weight)
Shucked Weight : 껍질을 제외한 무게 (weight of meat only)
Viscra Weight : 내장 무게 (gut weight, after bleeding)
Shell Weight : 껍질 무게 (weight after being dried)
------------------------------------------------------
```

In [1]:
import os
import numpy as np
import pandas as pd

In [2]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

train.drop('id', axis=1, inplace=True)
test.drop('id', axis=1, inplace=True)

train_X = train.drop('Target', axis=1)
train_y = train.Target

In [3]:
train_X = pd.get_dummies(data = train_X, columns = ['Gender'], prefix = 'Gender')
test = pd.get_dummies(data = test, columns = ['Gender'], prefix = 'Gender')

In [4]:
train_X

,Lenght,Diameter,Height,Whole Weight,Shucked Weight,Viscra Weight,Shell Weight,Gender_F,Gender_I,Gender_M
0,0.605,0.470,0.115,1.1140,0.3925,0.2910,0.3100,0,0,1
1,0.430,0.315,0.095,0.3780,0.1750,0.0800,0.1045,0,1,0
2,0.580,0.490,0.195,1.3165,0.5305,0.2540,0.4100,0,1,0
3,0.535,0.405,0.175,1.2705,0.5480,0.3265,0.3370,0,0,1
4,0.310,0.235,0.090,0.1270,0.0480,0.0310,0.0400,0,1,0
...,...,...,...,...,...,...,...,...,...,...
1248,0.190,0.145,0.040,0.0380,0.0165,0.0065,0.0150,0,1,0
1249,0.395,0.310,0.085,0.3170,0.1530,0.0505,0.0935,0,1,0
1250,0.525,0.410,0.115,0.7745,0.4160,0.1630,0.1800,1,0,0
1251,0.445,0.335,0.110,0.4355,0.2025,0.1095,0.1195,1,0,0


In [5]:
# 전복 전체 무게 중 불필요한 물질들의 값 ( 이물질 / 핏물 등)
foreign_body = train_X['Whole Weight'] - (train_X['Shucked Weight'] + train_X['Viscra Weight'] + train_X['Shell Weight'])

In [6]:
foreign_body

0       0.1205
1       0.0185
2       0.1220
3       0.0590
4       0.0080
         ...  
1248    0.0000
1249    0.0200
1250    0.0155
1251    0.0040
1252    0.1965
Length: 1253, dtype: float64

In [7]:
foreign_body[47] # 음수??

-0.06749999999999998

In [9]:
# 전복 전체 무게 대비 불필요한 물질들의 값 ( 이물질 / 핏물 등)
foreign_body = train_X['Whole Weight'] - (train_X['Shucked Weight'] + train_X['Viscra Weight'] + train_X['Shell Weight'])
train_X['foreign body'] = foreign_body
train_X.loc[train_X['foreign body'] < 0 , 'foreign body'] = 0

In [10]:
# 테스트셋에도 적용
foreign_body = test['Whole Weight'] - (test['Shucked Weight'] + test['Viscra Weight'] + test['Shell Weight'])
test['foreign body'] = foreign_body
test.loc[test['foreign body'] < 0 , 'foreign body'] = 0

In [11]:
# 껍질의 넓이 ( a * b * π)
area = 0.5 * train_X['Lenght'] * 0.5 * train_X['Diameter'] * np.pi
train_X['Area'] = area

In [12]:
# 껍질의 넓이 
area = 0.5 * test['Lenght'] * 0.5 * test['Diameter'] * np.pi
test['Area'] = area

In [13]:
# 껍질의 둘레 (근사) ( 2π*(0.5 * √(a^2 + b^2)))
perimeter = np.pi * np.sqrt(0.5 * ((train_X['Lenght'] ** 2) + (train_X['Diameter'] ** 2)))
train_X['Perimeter'] = perimeter

In [14]:
# 껍질의 둘레
perimeter = np.pi * np.sqrt(0.5 * ((test['Lenght'] ** 2) + (test['Diameter'] ** 2)))
test['Perimeter'] = perimeter

In [15]:
from tensorflow.keras.layers import *
from tensorflow.keras.models import *
from tensorflow.keras.optimizers import *
from tensorflow.keras.callbacks import *

In [16]:
# 딥러닝 모델 선언
model = Sequential()
model.add(Dense(16, input_dim=len(train_X.columns), activation='elu'))
model.add(Dense(32, activation='elu'))    
model.add(Dense(64, activation='elu'))  
model.add(Dropout(0.5))  
model.add(Dense(32, activation='elu'))
model.add(Dense(16, activation='elu'))
model.add(Dense(1))


model.compile(loss='mean_absolute_error',
              optimizer='Nadam', 
              metrics=['mae'])

# 모델 저장 폴더 만들기
MODEL_DIR = './model/'
if not os.path.exists(MODEL_DIR):
    os.mkdir(MODEL_DIR)

modelpath = "./model/{epoch:02d}-{val_loss:.4f}.hdf5"

# 모델 업데이트 및 저장
cp = ModelCheckpoint(filepath=modelpath, monitor='val_mae', verbose=0, save_best_only=True, mode = 'min')

# 학습 자동 중단 설정
es = EarlyStopping(monitor='val_mae', patience=50, mode='min')

rlrp = ReduceLROnPlateau(monitor='val_mae', factor=0.2, patience=40, mode='min')

# 모델 학습
model.fit(train_X, train_y, validation_split=0.3, epochs=1000, batch_size=32, verbose=1, callbacks=[es, cp, rlrp])

Epoch 1/1000
28/28 [==============================] - 1s 9ms/step - loss: 6.0431 - mae: 6.0431 - val_loss: 2.0536 - val_mae: 2.0536 - lr: 0.0010
Epoch 2/1000
28/28 [==============================] - 0s 2ms/step - loss: 2.1157 - mae: 2.1157 - val_loss: 1.9256 - val_mae: 1.9256 - lr: 0.0010
Epoch 3/1000
28/28 [==============================] - 0s 3ms/step - loss: 2.0257 - mae: 2.0257 - val_loss: 1.9054 - val_mae: 1.9054 - lr: 0.0010
Epoch 4/1000
28/28 [==============================] - 0s 3ms/step - loss: 1.9392 - mae: 1.9392 - val_loss: 1.9032 - val_mae: 1.9032 - lr: 0.0010
Epoch 5/1000
28/28 [==============================] - 0s 3ms/step - loss: 1.8962 - mae: 1.8962 - val_loss: 1.8910 - val_mae: 1.8910 - lr: 0.0010
Epoch 6/1000
28/28 [==============================] - 0s 2ms/step - loss: 1.8416 - mae: 1.8416 - val_loss: 1.9017 - val_mae: 1.9017 - lr: 0.0010
Epoch 7/1000
28/28 [==============================] - 0s 2ms/step - loss: 1.8365 - mae: 1.8365 - val_loss: 1.9299 - val_mae: 1.929

28/28 [==============================] - 0s 2ms/step - loss: 1.5081 - mae: 1.5081 - val_loss: 1.6918 - val_mae: 1.6918 - lr: 0.0010
Epoch 58/1000
28/28 [==============================] - 0s 2ms/step - loss: 1.5271 - mae: 1.5271 - val_loss: 1.7083 - val_mae: 1.7083 - lr: 0.0010
Epoch 59/1000
28/28 [==============================] - 0s 2ms/step - loss: 1.5325 - mae: 1.5325 - val_loss: 1.6624 - val_mae: 1.6624 - lr: 0.0010
Epoch 60/1000
28/28 [==============================] - 0s 2ms/step - loss: 1.5325 - mae: 1.5325 - val_loss: 1.6629 - val_mae: 1.6629 - lr: 0.0010
Epoch 61/1000
28/28 [==============================] - 0s 2ms/step - loss: 1.5103 - mae: 1.5103 - val_loss: 1.7916 - val_mae: 1.7916 - lr: 0.0010
Epoch 62/1000
28/28 [==============================] - 0s 2ms/step - loss: 1.5118 - mae: 1.5118 - val_loss: 1.7131 - val_mae: 1.7131 - lr: 0.0010
Epoch 63/1000
28/28 [==============================] - 0s 2ms/step - loss: 1.5134 - mae: 1.5134 - val_loss: 1.7817 - val_mae: 1.7817 - lr:

28/28 [==============================] - 0s 2ms/step - loss: 1.4806 - mae: 1.4806 - val_loss: 1.6248 - val_mae: 1.6248 - lr: 0.0010
Epoch 114/1000
28/28 [==============================] - 0s 2ms/step - loss: 1.4668 - mae: 1.4668 - val_loss: 1.6238 - val_mae: 1.6238 - lr: 0.0010
Epoch 115/1000
28/28 [==============================] - 0s 2ms/step - loss: 1.4848 - mae: 1.4848 - val_loss: 1.6199 - val_mae: 1.6199 - lr: 0.0010
Epoch 116/1000
28/28 [==============================] - 0s 2ms/step - loss: 1.4656 - mae: 1.4656 - val_loss: 1.6327 - val_mae: 1.6327 - lr: 0.0010
Epoch 117/1000
28/28 [==============================] - 0s 2ms/step - loss: 1.4725 - mae: 1.4725 - val_loss: 1.6238 - val_mae: 1.6238 - lr: 0.0010
Epoch 118/1000
28/28 [==============================] - 0s 3ms/step - loss: 1.4691 - mae: 1.4691 - val_loss: 1.6125 - val_mae: 1.6125 - lr: 0.0010
Epoch 119/1000
28/28 [==============================] - 0s 2ms/step - loss: 1.4715 - mae: 1.4715 - val_loss: 1.6650 - val_mae: 1.6650

Epoch 169/1000
28/28 [==============================] - 0s 2ms/step - loss: 1.4374 - mae: 1.4374 - val_loss: 1.5602 - val_mae: 1.5602 - lr: 0.0010
Epoch 170/1000
28/28 [==============================] - 0s 2ms/step - loss: 1.4227 - mae: 1.4227 - val_loss: 1.5899 - val_mae: 1.5899 - lr: 0.0010
Epoch 171/1000
28/28 [==============================] - 0s 2ms/step - loss: 1.4294 - mae: 1.4294 - val_loss: 1.5685 - val_mae: 1.5685 - lr: 0.0010
Epoch 172/1000
28/28 [==============================] - 0s 2ms/step - loss: 1.4349 - mae: 1.4349 - val_loss: 1.5730 - val_mae: 1.5730 - lr: 0.0010
Epoch 173/1000
28/28 [==============================] - 0s 2ms/step - loss: 1.4286 - mae: 1.4286 - val_loss: 1.5610 - val_mae: 1.5610 - lr: 0.0010
Epoch 174/1000
28/28 [==============================] - 0s 2ms/step - loss: 1.4523 - mae: 1.4523 - val_loss: 1.5593 - val_mae: 1.5593 - lr: 0.0010
Epoch 175/1000
28/28 [==============================] - 0s 2ms/step - loss: 1.4172 - mae: 1.4172 - val_loss: 1.5650 - 

Epoch 225/1000
28/28 [==============================] - 0s 2ms/step - loss: 1.4364 - mae: 1.4364 - val_loss: 1.5557 - val_mae: 1.5557 - lr: 0.0010
Epoch 226/1000
28/28 [==============================] - 0s 2ms/step - loss: 1.4161 - mae: 1.4161 - val_loss: 1.5627 - val_mae: 1.5627 - lr: 0.0010
Epoch 227/1000
28/28 [==============================] - 0s 2ms/step - loss: 1.4250 - mae: 1.4250 - val_loss: 1.5551 - val_mae: 1.5551 - lr: 0.0010
Epoch 228/1000
28/28 [==============================] - 0s 2ms/step - loss: 1.4123 - mae: 1.4123 - val_loss: 1.5654 - val_mae: 1.5654 - lr: 0.0010
Epoch 229/1000
28/28 [==============================] - 0s 2ms/step - loss: 1.4078 - mae: 1.4078 - val_loss: 1.5557 - val_mae: 1.5557 - lr: 0.0010
Epoch 230/1000
28/28 [==============================] - 0s 2ms/step - loss: 1.4139 - mae: 1.4139 - val_loss: 1.5400 - val_mae: 1.5400 - lr: 0.0010
Epoch 231/1000
28/28 [==============================] - 0s 2ms/step - loss: 1.4164 - mae: 1.4164 - val_loss: 1.5591 - 

28/28 [==============================] - 0s 2ms/step - loss: 1.3707 - mae: 1.3707 - val_loss: 1.5394 - val_mae: 1.5394 - lr: 2.0000e-04
Epoch 281/1000
28/28 [==============================] - 0s 2ms/step - loss: 1.4047 - mae: 1.4047 - val_loss: 1.5355 - val_mae: 1.5355 - lr: 2.0000e-04
Epoch 282/1000
28/28 [==============================] - 0s 2ms/step - loss: 1.3780 - mae: 1.3780 - val_loss: 1.5356 - val_mae: 1.5356 - lr: 2.0000e-04


In [17]:
Y_prediction = model.predict(test).flatten()

In [18]:
Y_prediction

array([10.3374815, 14.379316 ,  5.3089848, ...,  9.090191 ,  8.77253  ,
       12.030766 ], dtype=float32)

In [ ]:
submission = pd.read_csv('sample_submission.csv')

submission['Target'] = np.round(Y_prediction)

submission

submission.to_csv('submission.csv',index=False)